# Data Acquisition

In [3]:
!pip install pandas matplotlib

**Diabetic datasets Preview**

In [7]:
import pandas as pd

# Load the diabetes dataset
diabetes_data = pd.read_csv('/content/Diabetes_Metrics_With_State.csv')
# Display the first few rows of the diabetes data
print("\nDiabetes Data Preview:")
print(diabetes_data.head())

# Check for general information including null values in the census,county,diabetes data
print("\nDiabetes Data Preview:")
print(diabetes_data.info())

print("\nDiabetic Missing Values in Diabetes Data:")
print(diabetes_data.isnull().sum())


Diabetes Data Preview:
   Id  Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0   1            6      148             72             35        0  33.6   
1   2            1       85             66             29        0  26.6   
2   3            8      183             64              0        0  23.3   
3   4            1       89             66             23       94  28.1   
4   5            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome         State  
0                     0.627   50        1  PENNSYLVANIA  
1                     0.351   31        0   PUERTO RICO  
2                     0.672   32        1        NEVADA  
3                     0.167   21        0       INDIANA  
4                     2.288   33        1     TENNESSEE  

Diabetes Data Preview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2768 entries, 0 to 2767
Data columns (total 11 columns):
 #   Column                    Non-Null 

**Census Datasets Preview**

In [6]:
import pandas as pd

# Load the datasets
census_data = pd.read_csv('/content/acs2017_census_tract_data.csv')

# Display the first few rows of the census data
print("Census Data Preview:")
print(census_data.head())

# Check for general information including null values in the census,county data
print("\nCensus Data Information:")
print(census_data.info())

# Summarize missing values in both datasets
print("\nMissing Values in Census Data:")
print(census_data.isnull().sum())

Census Data Preview:
      TractId    State          County  TotalPop   Men  Women  Hispanic  \
0  1001020100  Alabama  Autauga County      1845   899    946       2.4   
1  1001020200  Alabama  Autauga County      2172  1167   1005       1.1   
2  1001020300  Alabama  Autauga County      3385  1533   1852       8.0   
3  1001020400  Alabama  Autauga County      4267  2001   2266       9.6   
4  1001020500  Alabama  Autauga County      9965  5054   4911       0.9   

   White  Black  Native  ...  Walk  OtherTransp  WorkAtHome  MeanCommute  \
0   86.3    5.2     0.0  ...   0.5          0.0         2.1         24.5   
1   41.6   54.5     0.0  ...   0.0          0.5         0.0         22.2   
2   61.4   26.5     0.6  ...   1.0          0.8         1.5         23.1   
3   80.3    7.1     0.5  ...   1.5          2.9         2.1         25.9   
4   77.5   16.4     0.0  ...   0.8          0.3         0.7         21.0   

   Employed  PrivateWork  PublicWork  SelfEmployed  FamilyWork  Unemplo

# Data Preprocessing

In [10]:
# Strip whitespace and convert to uppercase for merging
census_data['State'] = census_data['State'].str.strip().str.upper()
diabetes_data['State'] = diabetes_data['State'].str.strip().str.upper()

**Check for Missing Values**

In [11]:
# Fill missing values with median only for numeric columns
numeric_cols = census_data.select_dtypes(include='number').columns
census_data[numeric_cols] = census_data[numeric_cols].fillna(census_data[numeric_cols].median())

**Aggregate Census Data by State**

In [12]:
census_state_agg = census_data.groupby('State').median(numeric_only=True).reset_index()

**Merge on State**

In [13]:
merged_data = pd.merge(diabetes_data, census_state_agg, on='State', how='inner')

**Drop Duplicates or Unused Columns**

In [14]:
columns_of_interest = ['State', 'Glucose', 'BloodPressure', 'BMI', 'Age', 'Outcome',
                       'Income', 'Poverty', 'MeanCommute', 'Employed', 'Unemployment']
merged_data = merged_data[columns_of_interest]

In [15]:
print(merged_data.isnull().sum())
print(merged_data.describe())

State            0
Glucose          0
BloodPressure    0
BMI              0
Age              0
Outcome          0
Income           0
Poverty          0
MeanCommute      0
Employed         0
Unemployment     0
dtype: int64
           Glucose  BloodPressure          BMI          Age      Outcome  \
count  2768.000000    2768.000000  2768.000000  2768.000000  2768.000000   
mean    121.102601      69.134393    32.137392    33.132225     0.343931   
std      32.036508      19.231438     8.076127    11.777230     0.475104   
min       0.000000       0.000000     0.000000    21.000000     0.000000   
25%      99.000000      62.000000    27.300000    24.000000     0.000000   
50%     117.000000      72.000000    32.200000    29.000000     0.000000   
75%     141.000000      80.000000    36.625000    40.000000     1.000000   
max     199.000000     122.000000    80.600000    81.000000     1.000000   

             Income      Poverty  MeanCommute     Employed  Unemployment  
count   2768.00000

# Data Merging

In [17]:
import pandas as pd

# Load datasets
diabetes_data = pd.read_csv('/content/Diabetes_Metrics_With_State.csv')
census_data = pd.read_csv('/content/acs2017_census_tract_data.csv')

# Convert 'State' columns to uppercase to ensure matching
diabetes_data['State'] = diabetes_data['State'].str.upper()
census_data['State'] = census_data['State'].str.upper()

# Aggregate census data at the state level (mean values)
census_state_summary = census_data.groupby('State')[['Income', 'Poverty', 'MeanCommute', 'Employed', 'Unemployment']].mean().reset_index()

# Merge diabetes and summarized census data on State
merged_data = pd.merge(diabetes_data, census_state_summary, on='State', how='inner')

# Select final columns
selected_columns = [
    'State', 'Glucose', 'BloodPressure', 'BMI', 'Age', 'Outcome',
    'Income', 'Poverty', 'MeanCommute', 'Employed', 'Unemployment'
]

viz_data = merged_data[selected_columns]

# Save to CSV
viz_data.to_csv('/content/merged_dataset.csv', index=False)

print("✅ Visualization-ready dataset created successfully!")
print(viz_data.head())

✅ Visualization-ready dataset created successfully!
          State  Glucose  BloodPressure   BMI  Age  Outcome        Income  \
0  PENNSYLVANIA      148             72  33.6   50        1  59459.747643   
1   PUERTO RICO       85             66  26.6   31        0  21206.667429   
2        NEVADA      183             64  23.3   32        1  58820.623894   
3       INDIANA       89             66  28.1   21        0  51832.129333   
4     TENNESSEE      137             40  43.1   33        1  50463.940857   

     Poverty  MeanCommute     Employed  Unemployment  
0  14.420763    26.470801  1894.647918      7.044292  
1  46.247458    28.281087  1100.673016     19.011964  
2  14.627434    23.829056  1952.486172      8.348525  
3  16.995216    23.149035  2067.700199      6.903254  
4  18.219499    24.576626  2001.743487      7.320365  
